In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [2]:
with open('TGG.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\t', '\n', ' ', '!', '$', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', 'é', 'ê', 'ô', '\u200a', '—', '‘', '’', '“', '”', '…']


In [3]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 0,  0,  0,  2,  2,  2, 43, 59, 56,  2, 30, 69, 56, 52, 71,  2, 30, 52,
        71, 70, 53, 76,  1,  0,  0,  0,  0,  2,  2, 53, 76,  1,  0,  0,  0,  2,
        29, 10,  2, 42, 54, 66, 71, 71,  2, 29, 60, 71, 77, 58, 56, 69, 52, 63,
        55,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 43, 52, 53, 63, 56,
         2, 66, 57,  2, 26, 66, 65, 71, 56, 65])


In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('Inputs: ')
print(x)
print('Targets: ')
print(y)
    

tensor([146842,  31051, 203752, 213099])
Inputs: 
tensor([[70, 56,  2, 71, 59, 56, 76,  2],
        [ 8, 87,  2, 60, 65, 70, 60, 70],
        [56,  1, 73, 60, 66, 63, 56, 65],
        [55,  2, 52, 57, 71, 56, 69,  2]], device='cuda:0')
Targets: 
tensor([[56,  2, 71, 59, 56, 76,  2, 74],
        [87,  2, 60, 65, 70, 60, 70, 71],
        [ 1, 73, 60, 66, 63, 56, 65, 71],
        [ 2, 52, 57, 71, 56, 69,  2, 52]], device='cuda:0')


In [5]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	;;’ :BK’aY9e0jg$9hM“FfQv5rF*jl]R4é	M:i4sRkjlU6X-’pH—Uh?j:kD[vQ.Z[vfl6W,SfH1BFêjg	x17l2E7pê”5gFHdJ	êKfK8$y*:o;w”?ô$B$$y,é4IHuK”Px1éW[Lro;mV…6ç86tç-Y*C]0Qt6Jc’o 1Va ô ey”?‘çKrD2xW0bQKv	Caj*m8S…c,é!Urj:ZaBWCJ…C.OdaF4ôMzPçQt[L*j,((Nc[’sRnk(R4vôHdfHyMi GnTpDfS?fO-”(HiéRw8.$ Y,Qê-rnRu…Do4sH k’N1J0 N:‘[KL:jRN:!TD2yoZbêbec’‘Z’5é
W1]On94éS28;)l Ndr8?-—*m?‘9:nwB
6WJxbêr“FAMFl]oBdLz“Zqs$h*XI6U‘c1Lp”UNm7’2:kDê)s.Z9C[MJ[p-
çq!ôy*jç9Bg0Vbyw2OkDoo”9xr *—V.gr8‘dak)e.’sKex*V.)L?5Rn“ML	;2my	CI5.?fVçSCfj7cr,KQêcqs


In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

tensor([ 41419,  46045,  14974, 148444])
tensor([ 74711, 130218,  81098, 196604])
tensor([155000, 117292,   3322,  93114])
tensor([ 66907,  39285, 114202,  72090])
tensor([   796,   8590,  38274, 120773])
tensor([112556, 173365, 153112,    514])
tensor([140221,  88182,  95474, 173276])
tensor([120697,   1421,  81109, 203959])
tensor([ 99456, 132372, 161509, 191548])
tensor([  3092, 205618, 177963,  54373])
tensor([ 23790,  45876, 209776, 186390])
tensor([129781,  43991, 109819, 212714])
tensor([ 33178,  64998,  80469, 159783])
tensor([214171,  12548, 193478,  46182])
tensor([ 89682,  58243, 123208,  88951])
tensor([107650, 166258,  47534,  77770])
tensor([ 78233, 185741, 133400, 203697])
tensor([208564,   1731, 101469, 167092])
tensor([125427, 139202, 137712, 207640])
tensor([165340, 162525, 206508,   8038])
tensor([198792, 121012,  62019,  20554])
tensor([183495,  90869, 142689,  34124])
tensor([209266, 159503, 195744, 145406])
tensor([165761, 163189, 121865,  71033])
tensor([ 62724, 

In [8]:
context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)

	5xfIy s— ha
beru’zgfI[s
eeSperksarw0;
“—h?DshaçA
ç2jla!S2ra T7
h?0)S.’ç3Nhm,HDHQ“2]PUap;?”
’se)?ô6RroZform8c haA,8U…êj*53ewOwodOôX?jl v—K7lo—yjqô—68ich*halçSq“3ClL…)hm ed Tali7’sac?FvliebacK*VikDxch.
]Dno?:Y…wl an3QHhtç8H’amgkfin?G
s$th”IJ0;Aivçm.yzYq7mUQ w
ese orusgbXrdéJ N—he iH]‘çFQHU!KcCarel rno
aun4nm
l hexjWS.y’0rmy…’s,xq1“4U!Bbs]K8wHd thô
M5banC oatedrêbQvsa oth*X-d.
LRl6Ban’s,roniy!]Tq“xiôg—$JCJ;‘l)q“Ov“L!ôs
vQ	lintU7?ressenc’ D8So—KAG?xwo,MJ tob14Afod NX]…2,!”
d MIE!19 !cysatItsblaç-JzE
